In [1]:
import tensorflow as tf
import keras
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


In [5]:
# Fetch data
today=datetime.today().strftime('%Y-%m-%d')
data = yf.download('QQQ', start='2020-01-01', end=today)

# Display the first few rows of the dataframe
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,214.399994,216.160004,213.979996,216.160004,209.638062,30969400
2020-01-03,213.300003,215.470001,213.279999,214.179993,207.717773,27518900
2020-01-06,212.500000,215.589996,212.240005,215.559998,209.056168,21655300
2020-01-07,215.639999,216.139999,214.850006,215.529999,209.027039,22139300
2020-01-08,215.500000,218.139999,215.160004,217.149994,210.598160,26397300
...,...,...,...,...,...,...
2025-01-28,515.219971,523.000000,511.779999,521.809998,521.809998,33194200
2025-01-29,522.460022,522.590027,516.900024,520.830017,520.830017,26649000
2025-01-30,523.710022,526.099976,518.210022,523.049988,523.049988,27431300


# create target

In [7]:
# % chagne from prior close
data["daily_change_%"]=data["Close"].pct_change()*100
# % chagne current open-close
data["curday_change_%"]=(data["Close"]-data["Open"])/data["Open"]*100

In [9]:
data

,Open,High,Low,Close,Adj Close,Volume,daily_change_%,curday_change_%
Date,,,,,,,,
2020-01-02,214.399994,216.160004,213.979996,216.160004,209.638062,30969400,NaN,0.820900
2020-01-03,213.300003,215.470001,213.279999,214.179993,207.717773,27518900,-0.915993,0.412560
2020-01-06,212.500000,215.589996,212.240005,215.559998,209.056168,21655300,0.644320,1.439999
2020-01-07,215.639999,216.139999,214.850006,215.529999,209.027039,22139300,-0.013917,-0.051011
2020-01-08,215.500000,218.139999,215.160004,217.149994,210.598160,26397300,0.751633,0.765658
...,...,...,...,...,...,...,...,...
2025-01-28,515.219971,523.000000,511.779999,521.809998,521.809998,33194200,1.477991,1.279071
2025-01-29,522.460022,522.590027,516.900024,520.830017,520.830017,26649000,-0.187804,-0.311987
2025-01-30,523.710022,526.099976,518.210022,523.049988,523.049988,27431300,0.426237,-0.126030


# Data Preprocessing and Preparation
handling missing values, normalizing or scaling data, and potentially creating additional features, like moving averages or percentage changes

In [6]:
# Checking for missing values
data.isnull().sum()

# Filling missing values, it takes the last non-missing value from the previous row and fills it in.
# data.fillna(method='ffill', inplace=True)

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

use scaler if needed

In [14]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0,1))
# data_scaled = scaler.fit_transform(data['Close'].values.reshape(-1,1))
# data_scaled

array([[0.12703663],
       [0.12166886],
       [0.12541003],
       ...,
       [0.95900996],
       [0.95694959],
       [0.9456177 ]])

In [15]:
#Keras & Scikit-learn expect 2D arrays for training
data_scaled=data["curday_change_%"].values.reshape(-1,1)
data_scaled

array([[ 0.8209001 ],
       [ 0.41255959],
       [ 1.43999885],
       ...,
       [-0.12603047],
       [-0.87869222],
       [ 0.90365842]])

In [16]:
# setting x and y, train_test
X = []
y = []

for i in range(60, len(data_scaled)):
    X.append(data_scaled[i-60:i, 0])
    y.append(data_scaled[i, 0])
#x as feature- 60 prious data points? y- the next day price close as the target?

In [17]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshaping Data for LSTM
reshape our data into a 3D format [samples, time steps, features] required by LSTM layers

In [18]:
# X_train.shape[0]: Represents the number of samples in the dataset, often referred to as the batch size.
# X_train.shape[1]: Denotes the number of time steps in each sample.
# 1: Indicates that there is a single feature per time step, only close were selected
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Building the LSTM with Attention Model



In [19]:
#Creating LSTM Layers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply, Flatten, Lambda, Layer,BatchNormalization


model = Sequential()

# Adding LSTM layers with return_sequences=True,stacked LSTM architecture, 2 layers
# return_sequences=True is crucial in the first layers to ensure the output includes sequences
#LSTM layer outputs data in shape (batch_size, timesteps, features
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))



C:\Users\chenl\anaconda3\envs\notebook\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Integrating the Attention Mechanism
enhance the model’s ability to focus on relevant time steps

In [20]:

# Custom Attention Layer (Fix for Lambda Issue)
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.attention = AdditiveAttention()

    def call(self, inputs):
        attention_output = self.attention([inputs, inputs])  # Self-attention
        return Multiply()([inputs, attention_output])

# Define Model
model = Sequential()

# First LSTM Layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))

# Second LSTM Layer
model.add(LSTM(units=50, return_sequences=True))

# Apply Custom Attention Layer
model.add(AttentionLayer())

# Flatten Before Final Dense Layer
model.add(Flatten())

# Add Dropout for Regularization
model.add(Dropout(0.2))

# Add Batch Normalization
model.add(BatchNormalization())

# Final Dense Layer for Stock Price Prediction, 1 means next day price, change to x if want predict x days in future
model.add(Dense(1))

# Compile the Model
model.compile(optimizer='adam', loss='mean_squared_error')

# Show Model Summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 60, 50)         │            50 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3000)           │        12,000 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         3,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,651 (178.32 KB)

 Trainable params: 39,651 (154.89 KB)

 Non-trainable params: 6,000 (23.44 KB)

Total params (33,601) → The total number of parameters (weights + biases) in your model.


Trainable params (33,601) → These are the parameters that get updated during training (i.e., all layers are trainable).


Non-trainable params (0) → There are no frozen layers (e.g., pre-trained layers from another model).

# Training the Model





In [21]:
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - loss: 1.6573 - val_loss: 0.7177
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 1.7613 - val_loss: 0.7226
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 1.7058 - val_loss: 0.7213
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 1.5238 - val_loss: 0.7274
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 1.7582 - val_loss: 0.7185
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 1.7529 - val_loss: 0.7171
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 1.6628 - val_loss: 0.7179
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 1.7945 - val_loss: 0.7503
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 1.7056 - val_loss: 0.7176
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 1.6699 - val_loss: 0.7238
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 1.4350 - val_loss: 0.7538
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step

In [33]:
# from keras.callbacks import EarlyStopping
# # stops training when the model’s performance on the validation set starts to degrade.
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=[early_stopping])


# option ohter call back

In [23]:
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

# # Callback to save the model periodically
# model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# # Callback to reduce learning rate when a metric has stopped improving
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# # Callback for TensorBoard
# tensorboard = TensorBoard(log_dir='./logs')

# # Callback to log details to a CSV file
# csv_logger = CSVLogger('training_log.csv')

# # Combining all callbacks
# callbacks_list = [early_stopping, model_checkpoint, reduce_lr, tensorboard, csv_logger]

# # Fit the model with the callbacks
# history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=callbacks_list)

In [24]:
# Convert X_test and y_test to Numpy arrays if they are not already
X_test = np.array(X_test)
y_test = np.array(y_test)

# Ensure X_test is reshaped similarly to how X_train was reshaped
# This depends on how you preprocessed the training data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Now evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print("Test Loss: ", test_loss)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.8678
Test Loss:  2.2424967288970947


In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Making predictions
y_pred = model.predict(X_test)

# Calculating MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Mean Absolute Error: ", mae)
print("Root Mean Square Error: ", rmse)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step
Mean Absolute Error:  1.1273904362158826
Root Mean Square Error:  1.497496846139528


# use prediction 

In [32]:
# Fetching the latest 60 days of AAPL stock data
data = yf.download('QQQ', period='60d', interval='1d')
data["curday_change_%"]=(data["Close"]-data["Open"])/data["Open"]*100

# Selecting the 'Close' price and converting to numpy array
scaled_data = data["curday_change_%"].values.reshape(-1,1)

# Scaling the data
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(closing_prices.reshape(-1,1))

# Since we need the last 60 days to predict the next day, we reshape the data accordingly
X_latest = np.array([scaled_data[-60:].reshape(60)])

# Reshaping the data for the model (adding batch dimension)
X_latest = np.reshape(X_latest, (X_latest.shape[0], X_latest.shape[1], 1))

# Making predictions for the next 4 candles
predicted_stock_price = model.predict(X_latest)
# predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

print("Predicted Stock next day: ", predicted_stock_price)

[*********************100%%**********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Stock Prices for the next day:  [[1.1843278]]


In [27]:
# from datetime import datetime, timedelta

# def predict_stock_price(input_date):
#     # Check if the input date is a valid date format
#     try:
#         input_date = pd.to_datetime(input_date)
#     except ValueError:
#         print("Invalid Date Format. Please enter date in YYYY-MM-DD format.")
#         return

#     # Fetch data from yfinance
#     end_date = input_date
#     start_date = input_date - timedelta(days=90)  # Fetch more days to ensure we have 60 trading days
#     data = yf.download('AAPL', start=start_date, end=end_date)

#     if len(data) < 60:
#         print("Not enough historical data to make a prediction. Try an earlier date.")
#         return

#     # Prepare the data
#     closing_prices = data['Close'].values[-60:]  # Last 60 days
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaled_data = scaler.fit_transform(closing_prices.reshape(-1, 1))

#     # Make predictions
#     predicted_prices = []
#     current_batch = scaled_data.reshape(1, 60, 1)

#     for i in range(4):  # Predicting 4 days
#         next_prediction = model.predict(current_batch)
#         next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
#         current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
#         predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

#     # Output the predictions
#     for i, price in enumerate(predicted_prices, 1):
#         print(f"Day {i} prediction: {price}")

# # Example use
# user_input = input("Enter a date (YYYY-MM-DD) to predict stock for the next 4 days: ")
# predict_stock_price(user_input)

# next step, 
1. input more features?
2. create diff target: % change of the day (open to close, prior close to close), also try just classification for direction, and longer expect
3. tuning the basic model
4. adjust this LSTM model (the scaler)